In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
keyword_df = pd.read_pickle('../data/keywords.pk')
words_df = pd.read_pickle('../data/recwords.pk')

In [31]:
def lower_case_exc(something):
    if type(something)==str:
        return something.lower()
    else:
        return something

In [9]:
recipe_ids = list(set(keyword_df.index).intersection(set(words_df.index)))
keyword_df = keyword_df[keyword_df.index.isin(recipe_ids)]
words_df = words_df[words_df.index.isin(recipe_ids)]

In [32]:
for col in words_df.columns:
    words_df[col]=words_df[col].apply(lambda x:lower_case_exc(x))

In [54]:
words_df['Keywords_orig']=[keyword_df['Keywords'][recipe] for recipe in words_df.index]

In [35]:
categories = list({w for w in words_df.RecipeCategory if type(w)==str})

In [37]:
len(words_df.RecipeCategory.unique())

272

In [17]:
keyword_list_orig = []
for kwl in keyword_df.Keywords:
    keyword_list_orig+=list(kwl)

keyword_list_orig_nr = list(set(keyword_list_orig))

In [45]:
words_df.head()

,Name,Description,RecipeCategory
RecipeId,,,
38,low-fat berry blue frozen dessert,make and share this low-fat berry blue frozen ...,frozen desserts
40,best lemonade,this is from one of my first good house keepi...,beverages
42,cabbage soup,make and share this cabbage soup recipe from f...,vegetable
44,warm chicken a la king,i copied this one out of a friend's book so ma...,chicken
45,buttermilk pie with gingersnap crumb crust,make and share this buttermilk pie with ginger...,pie


In [38]:
extrakeywords=['pasta','chicken','rice','cheap','simple','burger','veggie','fried','sauce','soup','chocolate','pudding','taco','hummus','healthy',
               'cake','egg','italian','mushroom','casserole','french toast','pancake','waffle','salad','pie','noodle','ramen','pizza','greek']

In [40]:
keyword_list_new = list(set(keyword_list_orig_nr+extrakeywords+categories))

In [50]:
keywords_in_titles = []
for name in words_df.Name:
    keywords_in_titles.append([word for word in keyword_list_new if word in name])
words_df['KeywordsTitle']=keywords_in_titles

In [52]:
keywords_in_desc = []
for desc in words_df.Description:
    keywords_in_desc.append([word for word in keyword_list_new if word in desc])
words_df['KeywordsDesc']=keywords_in_desc

In [57]:
new_keyword_data = []
for rec in words_df.index:
    kws_new = []
    if type(words_df['RecipeCategory'][rec])==str:
        kws_new.append(words_df['RecipeCategory'][rec])
    for col in ['KeywordsTitle','KeywordsDesc','Keywords_orig']:
        kws_new+=list(words_df[col][rec])
    kws_new = list(set(kws_new))
    new_keyword_data.append(kws_new)
words_df['Keywords'] = new_keyword_data

In [61]:
words_df['Keywords_orig'].apply(len).mean()

5.70336575332915

In [64]:
words_df[['Name','KeywordsTitle','Keywords']].to_pickle('../data/keywords.pk')

In [3]:
keyword_df

,Name,KeywordsTitle,Keywords
RecipeId,,,
38,low-fat berry blue frozen dessert,[dessert],"[frozen desserts, healthy, freezer, dessert, s..."
40,best lemonade,[lemon],"[healthy, summer, lemon, beverages]"
42,cabbage soup,"[cabbage, soup]","[healthy, vegetable, winter, vegan, cabbage, e..."
44,warm chicken a la king,[chicken],"[noodle, pie, potato, rice, chicken, meat, pou..."
45,buttermilk pie with gingersnap crumb crust,[pie],"[pie, healthy, dessert, weeknight, oven]"
...,...,...,...
539182,cinnamony sweet moroccan orange salad,"[salad, moroccan, sweet]","[citrus, healthy, fruit, oranges, sweet, salad..."
539184,belgian chocolate-fudge sauce,"[chocolate, belgian, sauce]","[waffle, sauce, chocolate, sauces, belgian, de..."
539304,pork tenderloin with bacon-onion jam,[pork],"[pork, meat]"


In [4]:
keyword_list_orig = []
for kwl in keyword_df.Keywords:
    keyword_list_orig+=list(kwl)

keyword_list_orig_nr = list(set(keyword_list_orig))

In [5]:
len(keyword_list_orig_nr)

332

In [10]:
keyword_list_orig_nr

['household cleaner',
 'canadian',
 'macaroni and cheese',
 'guatemalan',
 'pudding',
 'scandinavian',
 'pasta shells',
 'cheap',
 'coconut',
 'medium grain rice',
 'crock pot slow cooker',
 'vietnamese',
 'native american',
 'candy',
 'southwestern u.s.',
 'peruvian',
 'one dish meal',
 'cake',
 'sudanese',
 'malaysian',
 'turkish',
 'tilapia',
 '< 15 mins',
 'summer dip',
 'orange roughy',
 'bath/beauty',
 'danish',
 'raspberries',
 'oysters',
 'beef crock pot',
 'puerto rican',
 'pumpkin',
 'taco',
 'cantonese',
 'savory',
 'from scratch',
 'lemon',
 'birthday',
 'goose',
 'south american',
 'peppers',
 'ham',
 'spicy',
 'greens',
 'canning',
 'quick breads',
 'palestinian',
 'broil/grill',
 'halloween',
 'collard greens',
 'steak',
 'egyptian',
 'quail',
 'yam/sweet potato',
 'chicken thigh & leg',
 'reynolds wrap contest',
 'lunch/snacks',
 'nigerian',
 'christmas',
 'breakfast eggs',
 'baking',
 'low protein',
 'pork crock pot',
 'egg',
 'swiss',
 'polynesian',
 'no cook',
 'past

In [7]:
rice_words = [w for w in keyword_list_orig_nr if 'rice' in w]


In [78]:
def region_check(word):
    for ending in ['ian','ino','can','lan']:
        if word[-3:]==ending:
            return True
    if word[-3:]=='ish' and word[-4]!='f':
        return True
    elif word[-3:]=='ese' and word[-4]!='e':
        return True
    elif 'west'in word:
        return True
    return False

regions = [w for w in keyword_list_orig_nr if region_check(w)]
regions+=['thai', 'new zealand','iraqi', 'creole', 'welsh','szechuan', 'hunan','korean',
          'european', 'icelandic', 'pennsylvania dutch','pakistani', 'honduran', 'dutch',
          'ecuadorean','chilean','czech','swiss','cuban', 'tex mex']

In [95]:
keyword_df['Regions'] = [list(set(kws).intersection(set(regions))) for kws in keyword_df.Keywords]

In [70]:
crockpotwords = [w for w in keyword_list_orig_nr if 'crock pot' in w]
potatowords = [w for w in keyword_list_orig_nr if 'potato' in w]
beefwords = [w for w in keyword_list_orig_nr if 'beef' in w]
chickenwords = [w for w in keyword_list_orig_nr if 'chicken' in w]
soupwords = [w for w in keyword_list_orig_nr if 'soup' in w]
beanwords = [w for w in keyword_list_orig_nr if 'bean' in w]
saucewords = [w for w in keyword_list_orig_nr if 'sauce' in w]
breadwords = [w for w in keyword_list_orig_nr if 'bread' in w]
breakfastwords = [w for w in keyword_list_orig_nr if 'breakfast' in w]
saucewords = [w for w in keyword_list_orig_nr if 'sauce' in w]

In [108]:
unsorted_keywords = [w for w in keyword_list_orig_nr if w not in 
                     chickenwords+rice_words+regions+crockpotwords+potatowords+beefwords+soupwords+beanwords+saucewords+fish+appliance_words]

In [84]:
keyword_df.loc[keyword_df.RegionKnown==0]

,Name,KeywordsTitle,Keywords,RegionKnown
RecipeId,,,,
38,low-fat berry blue frozen dessert,[dessert],"[frozen desserts, healthy, freezer, dessert, s...",0
40,best lemonade,[lemon],"[healthy, summer, lemon, beverages]",0
42,cabbage soup,"[cabbage, soup]","[healthy, vegetable, winter, vegan, cabbage, e...",0
44,warm chicken a la king,[chicken],"[noodle, pie, potato, rice, chicken, meat, pou...",0
45,buttermilk pie with gingersnap crumb crust,[pie],"[pie, healthy, dessert, weeknight, oven]",0
...,...,...,...,...
539095,the soul train,[],"[beverages, simple, cheese]",0
539102,mary pickford cocktail,[],"[cherries, beverages, healthy, simple, apple, ...",0
539175,garlicky flash zucchini,[],"[vegetable, easy, simple]",0


In [85]:
revs = pd.read_pickle('../data/reviewsred.pk')

In [89]:
revs = revs[revs['RecipeId'].isin(keyword_df.index)]

In [104]:
keyword_df[keyword_df.index.isin(noshellfishrecs)]

,Name,KeywordsTitle,Keywords,RegionKnown,Regions
RecipeId,,,,,
4009,linguine pescatore,[],"[mussels, european, no shell fish, squid]",1,[european]
15258,crumbed calamari,[],"[deep fried, sauce, no shell fish, squid]",0,[]
24858,calamari special,[],"[deep fried, no shell fish, squid, easy, asian]",1,[asian]
28537,gilroy garlic festival red sauce (for calamari...,[sauce],"[sauce, no shell fish, squid, pasta, stove top]",0,[]
31480,paella,[],"[pork, rice, spanish, no shell fish, chicken, ...",2,"[spanish, european]"
...,...,...,...,...,...
301969,ginger squid japanese style (ika shoga yaki),"[squid, japanese]","[australian, stove top, stir fry, no shell fis...",3,"[japanese, australian, asian]"
327719,"orzo with shrimp, feta cheese and white wine",[cheese],"[nuts, no shell fish, pasta, easy, cheese]",0,[]
343596,italian stuffed squid,"[squid, italian]","[no shell fish, italian, squid]",1,[italian]


In [101]:
noshellfishrecs= [r for r in keyword_df.index if 'no shell fish' in keyword_df['Keywords'][r]]

In [96]:
revs['Regions'] = [keyword_df['Regions'][recipe] for recipe in revs.RecipeId]

In [93]:
revs.loc[revs.RegKnown==0]

,UserId,RecipeId,Rating,RegKnown
ReviewId,,,,
7,1634,4384,4,0
17,2046,5221,4,0
19,2046,13307,5,0
23,2046,4684,5,0
25,2046,3431,5,0
...,...,...,...,...
2090281,843817,437232,5,0
2090287,2038463,26370,0,0
2090291,2002871210,80156,0,0


In [97]:
revs.loc[revs.RegKnown>0]

,UserId,RecipeId,Rating,RegKnown,Regions
ReviewId,,,,,
9,2046,4523,2,2,"[chinese, asian]"
13,1773,7435,5,1,[european]
22,2046,517,5,1,[european]
24,1773,278,4,1,[european]
47,2312,780,5,2,"[indian, asian]"
...,...,...,...,...,...
2090128,559493,214498,5,2,"[european, italian]"
2090199,128473,18458,5,1,[russian]
2090251,2038463,49088,5,1,[mexican]


In [98]:
unsorted_keywords

['household cleaner',
 'macaroni and cheese',
 'pudding',
 'pasta shells',
 'cheap',
 'coconut',
 'candy',
 'one dish meal',
 'cake',
 'tilapia',
 '< 15 mins',
 'summer dip',
 'orange roughy',
 'bath/beauty',
 'raspberries',
 'oysters',
 'pumpkin',
 'taco',
 'savory',
 'from scratch',
 'lemon',
 'birthday',
 'goose',
 'peppers',
 'ham',
 'spicy',
 'greens',
 'canning',
 'quick breads',
 'broil/grill',
 'halloween',
 'collard greens',
 'steak',
 'quail',
 'reynolds wrap contest',
 'lunch/snacks',
 'christmas',
 'breakfast eggs',
 'baking',
 'low protein',
 'egg',
 'no cook',
 'pasta',
 'manicotti',
 'pork',
 'spaghetti',
 'casserole',
 'free of...',
 'rabbit',
 'sweet',
 'pizza',
 'hanukkah',
 'small appliance',
 'mahi mahi',
 'penne',
 'soy/tofu',
 'for large groups holiday/event',
 'pie',
 'hummus',
 'cauliflower',
 'plums',
 'grapes',
 'mango',
 'steam',
 'strawberry',
 'mixer',
 'vegan',
 'brunch',
 'trout',
 'pheasant',
 'simple',
 'mussels',
 'onions',
 'refrigerator',
 'lactose f

In [143]:
appliance_words = ['pressure cooker','stove top','oven','broil/grill','microwave','bread machine']
bread = [w for w in keyword_list_orig_nr if 'bread' in w]
fish = ['tilapia','halibut','squid','tuna','oysters']+[w for w in unsorted_keywords if 'fish' in w]
bakedgoods = ['cake','chocolate chip cookies']
fruitveg = ['coconut','raspberries','pumpkin','lemon','tropical fruits','artichoke','peppers','collard greens','greens']
recipe_types =['pudding','macaroni and cheese','chutneys','savory pies']
beefwords+=['steak']
soupwords+=['gumbo']+[w for w in keyword_list_orig_nr if 'stew' in w]
porkwords = ['ham']+[w for w in keyword_list_orig_nr if 'pork' in w]
poultry_nonchicken = ['goose','quail']
events = ['christmas','chinese new year','labor day','thanksgiving','memorial day',"st. patrick's day"]
pastas = ['pasta shells','spaghetti','manicotti','pasta']
sortednew = appliance_words+fish+bakedgoods+fruitveg+recipe_types+beefwords+porkwords+poultry_nonchicken+soupwords+bread+events+pastas

In [144]:
unsorted_keywords = [w for w in unsorted_keywords if w not in sortednew and '<' not in w]

In [138]:
keyword_df[keyword_df.index.isin(noshellfishrecs)]

,Name,KeywordsTitle,Keywords,RegionKnown,Regions
RecipeId,,,,,
4009,linguine pescatore,[],"[mussels, european, no shell fish, squid]",1,[european]
15258,crumbed calamari,[],"[deep fried, sauce, no shell fish, squid]",0,[]
24858,calamari special,[],"[deep fried, no shell fish, squid, easy, asian]",1,[asian]
28537,gilroy garlic festival red sauce (for calamari...,[sauce],"[sauce, no shell fish, squid, pasta, stove top]",0,[]
31480,paella,[],"[pork, rice, spanish, no shell fish, chicken, ...",2,"[spanish, european]"
...,...,...,...,...,...
301969,ginger squid japanese style (ika shoga yaki),"[squid, japanese]","[australian, stove top, stir fry, no shell fis...",3,"[japanese, australian, asian]"
327719,"orzo with shrimp, feta cheese and white wine",[cheese],"[nuts, no shell fish, pasta, easy, cheese]",0,[]
343596,italian stuffed squid,"[squid, italian]","[no shell fish, italian, squid]",1,[italian]


In [139]:
[w for w in keyword_list_orig_nr if 'stew' in w]

['chicken stew', 'stew']

In [145]:
unsorted_keywords

['household cleaner',
 'cheap',
 'candy',
 'one dish meal',
 'summer dip',
 'orange roughy',
 'bath/beauty',
 'taco',
 'savory',
 'from scratch',
 'birthday',
 'spicy',
 'canning',
 'halloween',
 'reynolds wrap contest',
 'lunch/snacks',
 'breakfast eggs',
 'baking',
 'low protein',
 'egg',
 'no cook',
 'casserole',
 'free of...',
 'rabbit',
 'sweet',
 'pizza',
 'hanukkah',
 'small appliance',
 'mahi mahi',
 'penne',
 'soy/tofu',
 'for large groups holiday/event',
 'pie',
 'hummus',
 'cauliflower',
 'plums',
 'grapes',
 'mango',
 'steam',
 'strawberry',
 'mixer',
 'vegan',
 'brunch',
 'trout',
 'pheasant',
 'simple',
 'mussels',
 'onions',
 'refrigerator',
 'lactose free',
 'scones',
 'pineapple',
 'jellies',
 'gelatin',
 'curries',
 'burger',
 'ramadan',
 'spinach',
 'dessert',
 'french toast',
 'pasta elbow',
 'avocado',
 'german',
 'beverages',
 'cherries',
 'noodle',
 'tempeh',
 'oranges',
 'greek',
 'wild game',
 'cajun',
 'homeopathy/remedies',
 'high protein',
 'healthy',
 'smoo

In [141]:
len(unsorted_keywords)

168

In [135]:
pastas = ['pasta shells','spaghetti','manicotti','pasta']

In [142]:
'pasta' in keyword_list_orig_nr

True

In [1]:
regions

NameError: name 'regions' is not defined